In [1]:
!pip install --upgrade spleeter noisereduce librosa soundfile pydub tensorflow tensorflow-estimator

  Using cached tensorflow-2.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tensorflow_estimator-2.15.0-py2.py3-none-any.whl.metadata (1.3 kB)


In [ ]:
import os
import glob
import tempfile
from spleeter.separator import Separator
import noisereduce as nr
import librosa
import soundfile as sf
from pydub import AudioSegment
from pydub.effects import low_pass_filter, high_pass_filter
from tqdm import tqdm

def convert_to_wav(input_path, output_path):
    """Конвертирует аудиофайл в WAV формат"""
    try:
        audio = AudioSegment.from_file(input_path)
        audio.export(output_path, format="wav")
        return output_path
    except Exception as e:
        print(f"Error converting {input_path}: {str(e)}")
        return None

def separate_vocals(input_audio, output_path):
    """Отделяет вокал от музыки с помощью Spleeter"""
    try:
        separator = Separator('spleeter:2stems')
        separator.separate_to_file(input_audio, output_path)
        base_name = os.path.splitext(os.path.basename(input_audio))[0]
        return os.path.join(output_path, base_name, "vocals.wav")
    except Exception as e:
        print(f"Error separating vocals from {input_audio}: {str(e)}")
        return None

def clean_audio(input_path, output_path, noise_start=0, noise_end=1):
    """Очищает аудио от шумов"""
    try:
        y, sr = librosa.load(input_path, sr=None)
        noise_sample = y[int(noise_start * sr) : int(noise_end * sr)]
        cleaned_audio = nr.reduce_noise(y=y, sr=sr, y_noise=noise_sample, stationary=True, prop_decrease=1.0)
        sf.write(output_path, cleaned_audio, sr)
        return output_path
    except Exception as e:
        print(f"Error cleaning {input_path}: {str(e)}")
        return None

def postprocess_audio(input_path, output_path):
    """Постобработка аудио"""
    try:
        audio = AudioSegment.from_wav(input_path)
        filtered = high_pass_filter(audio, cutoff=100)
        filtered = low_pass_filter(filtered, cutoff=4000).normalize()
        filtered.export(output_path, format="wav")
        return output_path
    except Exception as e:
        print(f"Error postprocessing {input_path}: {str(e)}")
        return None

def process_single_audio(input_path, output_folder):
    """Обрабатывает один аудиофайл с полной очисткой временных файлов"""
    try:
        base_name = os.path.splitext(os.path.basename(input_path))[0]
        final_path = os.path.join(output_folder, f"{base_name}_clean.wav")

        with tempfile.TemporaryDirectory() as temp_dir:
            temp_wav = os.path.join(temp_dir, f"{base_name}.wav")
            if not convert_to_wav(input_path, temp_wav):
                return None

            vocals_path = separate_vocals(temp_wav, temp_dir)
            if not vocals_path or not os.path.exists(vocals_path):
                return None

            cleaned_path = os.path.join(temp_dir, f"cleaned_{base_name}.wav")
            if not clean_audio(vocals_path, cleaned_path):
                return None

            if not postprocess_audio(cleaned_path, final_path):
                return None

        return final_path
    except Exception as e:
        print(f"Error processing {input_path}: {str(e)}")
        return None

def process_audio_folder(input_folder, output_folder):
    """Обрабатывает все аудиофайлы в папке"""
    os.makedirs(output_folder, exist_ok=True)
    audio_files = []

    for ext in ['*.mp3', '*.wav', '*.WAV', '*.MP3']:
        audio_files.extend(glob.glob(os.path.join(input_folder, ext)))

    if not audio_files:
        print(f"No audio files found in {input_folder}")
        return []

    print(f"Found {len(audio_files)} audio files to process")
    processed_files = []

    for audio_file in tqdm(audio_files, desc="Processing audio files"):
        result = process_single_audio(audio_file, output_folder)
        if result:
            processed_files.append(result)

    print(f"\nProcessing complete! {len(processed_files)} files processed")
    print(f"Cleaned files saved to: {output_folder}")
    return processed_files

input_folder = "/content/input_audio"
output_folder = "/content/cleaned_audio"

os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

processed_files = process_audio_folder(input_folder, output_folder)

Found 2 audio files to process


Processing audio files:   0%|          | 0/2 [00:00<?, ?it/s]WARNING:tensorflow:From /usr/local/lib/python3.11/dist-packages/spleeter/separator.py:65: RunConfig.__init__ (from tensorflow_estimator.python.estimator.run_config) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.


INFO:spleeter:File /tmp/tmp5q8kxff5/222/vocals.wav written succesfully


INFO:spleeter:File /tmp/tmp5q8kxff5/222/vocals.wav written succesfully


INFO:spleeter:File /tmp/tmp5q8kxff5/222/accompaniment.wav written succesfully


...

INFO:spleeter:File /tmp/tmp0439jg10/111/vocals.wav written succesfully


INFO:spleeter:File /tmp/tmp0439jg10/111/vocals.wav written succesfully


INFO:spleeter:File /tmp/tmp0439jg10/111/accompaniment.wav written succesfully


INFO:spleeter:File /tmp/tmp0439jg10/111/accompaniment.wav written succesfully
Processing audio files: 100%|██████████| 2/2 [02:20<00:00, 70.20s/it]


Processing complete! 2 files processed
Cleaned files saved to: /content/cleaned_audio
